# Construcción del dataset

In [1]:
import pandas as pd
from mlforecast import MLForecast
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import numpy as np

from window_ops.rolling import rolling_mean
import optuna
from sklearn.model_selection import TimeSeriesSplit
from utilsforecast.feature_engineering import fourier

/media/nespina/Elements/maestriacd/austral/labo_3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

df = pd.read_csv('../data/sell-in.txt', sep='\t', encoding='utf-8')
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452


In [3]:
df_productos_predecir = pd.read_csv('../data/product_id_apredecir201912.txt', sep='\t', encoding='utf-8')
df_productos_predecir.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [4]:
df['periodo'].sort_values().unique()

array([201701, 201702, 201703, 201704, 201705, 201706, 201707, 201708,
       201709, 201710, 201711, 201712, 201801, 201802, 201803, 201804,
       201805, 201806, 201807, 201808, 201809, 201810, 201811, 201812,
       201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912])

In [5]:
df_pivot = df.pivot_table(
    index=['product_id', 'customer_id'],
    columns='periodo',
    values='tn',
    aggfunc='sum',
    fill_value=None
)
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None
df_pivot.head()

,product_id,customer_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,20001,10001,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,20001,10002,35.72806,6.79415,29.94128,22.81133,31.22847,47.57025,21.84874,17.08052,...,31.97079,55.41679,30.87299,144.07021,37.14616,NaN,72.08551,17.40806,45.61495,113.33165
2,20001,10003,143.49426,20.48319,137.87537,68.89292,135.12190,171.01785,64.66196,83.63410,...,170.89924,230.00152,1.84835,NaN,138.23391,162.07198,233.20532,76.00625,86.14415,102.27517
3,20001,10004,184.72927,104.03894,295.43924,247.65632,188.37819,195.02683,379.44270,237.16848,...,102.64484,91.67799,389.02653,66.71971,228.62366,96.11402,288.34205,324.96172,195.67828,34.64810
4,20001,10005,19.08407,5.17117,5.17117,0.86186,37.95546,19.08407,43.35049,67.53856,...,6.90049,22.18016,15.89578,NaN,8.25595,NaN,12.80400,17.13921,12.22149,19.60368


In [6]:
# Remove from df_pivot the products that are not in df_productos_predecir
df_pivot = df_pivot[df_pivot['product_id'].isin(df_productos_predecir['product_id'])]

In [7]:
# df_mlforecast = df_pivot[df_pivot['customer_id'] == 10001].copy()
df_mlforecast = df_pivot.copy()

In [8]:
df_mlforecast.head()

,product_id,customer_id,201701,201702,201703,201704,201705,201706,201707,201708,...,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
0,20001,10001,99.43861,198.84365,92.46537,13.29728,101.00563,128.04792,101.20711,43.33930,...,130.54927,364.37071,439.90647,65.92436,144.78714,33.63991,109.05244,176.02980,236.65556,180.21938
1,20001,10002,35.72806,6.79415,29.94128,22.81133,31.22847,47.57025,21.84874,17.08052,...,31.97079,55.41679,30.87299,144.07021,37.14616,NaN,72.08551,17.40806,45.61495,113.33165
2,20001,10003,143.49426,20.48319,137.87537,68.89292,135.12190,171.01785,64.66196,83.63410,...,170.89924,230.00152,1.84835,NaN,138.23391,162.07198,233.20532,76.00625,86.14415,102.27517
3,20001,10004,184.72927,104.03894,295.43924,247.65632,188.37819,195.02683,379.44270,237.16848,...,102.64484,91.67799,389.02653,66.71971,228.62366,96.11402,288.34205,324.96172,195.67828,34.64810
4,20001,10005,19.08407,5.17117,5.17117,0.86186,37.95546,19.08407,43.35049,67.53856,...,6.90049,22.18016,15.89578,NaN,8.25595,NaN,12.80400,17.13921,12.22149,19.60368


In [9]:
# --- PASO 1: TRANSFORMACIÓN DE DATOS A FORMATO LARGO ---
# Este es el formato conveniente que usaremos en ambos casos.
print("\n--- 1. Transformando datos a formato largo ---")
df_long = df_mlforecast.melt(
    id_vars=['product_id', 'customer_id'],
    var_name='periodo',
    value_name='y' # MLForecast usa 'y' como nombre de la variable objetivo
)



--- 1. Transformando datos a formato largo ---


In [10]:
df_long.head()

,product_id,customer_id,periodo,y
0,20001,10001,201701,99.43861
1,20001,10002,201701,35.72806
2,20001,10003,201701,143.49426
3,20001,10004,201701,184.72927
4,20001,10005,201701,19.08407


In [11]:
# Filtrar todos los DataFrames para conservar solo los registros con customer_id 10001
# df_long = df_long[df_long['customer_id'] == 10001]

In [12]:
df_long = df_long.fillna(0)
# Creamos las columnas que MLForecast requiere: unique_id, ds, y
df_long['unique_id'] = df_long['product_id'].astype(str) + "_" + df_long['customer_id'].astype(str)
df_long['ds'] = pd.to_datetime(df_long['periodo'], format='%Y%m')

# Seleccionamos y ordenamos las columnas finales
df_final = df_long[['unique_id', 'ds', 'y']].sort_values(by=['unique_id', 'ds']).reset_index(drop=True)
print("Formato de datos listo:")
df_final.head()

/tmp/ipykernel_695331/3730884332.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_long = df_long.fillna(0)


Formato de datos listo:


,unique_id,ds,y
0,20001_10001,2017-01-01,99.43861
1,20001_10001,2017-02-01,198.84365
2,20001_10001,2017-03-01,92.46537
3,20001_10001,2017-04-01,13.29728
4,20001_10001,2017-05-01,101.00563


In [14]:
# fcst = MLForecast(
#     models=[
#         LGBMRegressor(
#             random_state=42,
#             n_estimators=132,
#             learning_rate=0.0425038112097633,
#             num_leaves=83,
#             max_depth=3,
#             min_child_samples=18,
#             subsample=0.8823228364012079,
#             colsample_bytree=0.7618871571728139
#         )
#     ],
#     freq='MS',
#     lags=[1, 2, 3, 6, 12],
#     lag_transforms={
#         1: [(rolling_mean, 3), (rolling_mean, 6)],
#         3: [(rolling_mean, 3)]
#     },
#     date_features=['month', 'year', 'dayofweek']
# )

# fcst = MLForecast(
#     models=LGBMRegressor(random_state=42, n_estimators=100),
#     freq='MS',
#     lags=[1, 2, 3, 6, 12], # <-- Lag estacional de 12 meses
#     date_features=['month', 'quarter', 'year'], # <-- Features de fecha
#     target_transforms=[
#         fourier(
#             df=df_entrenamiento,
#             freq='M', 
#             season_length=12,
#             k=12)
#     ] # <-- Features de Fourier
# )

In [15]:


# 1. Creando features de Fourier correctamente
print("--- 1. Creando features de Fourier ---")
# La función fourier necesita la serie de fechas, no el DataFrame completo.
# Y devuelve un DataFrame, no una tupla.
fourier_features = fourier(df=df_final, freq='MS', season_length=12, k=12, time_col='ds')
df_final = pd.concat([df_final, fourier_features[0]], axis=1)
df_final = df_final.loc[:, ~df_final.columns.duplicated()] # Buena práctica

# 2. Dividir los datos
FECHA_CORTE = '2019-10-01'
df_entrenamiento = df_final[df_final['ds'] <= FECHA_CORTE]
df_validacion = df_final[df_final['ds'] > FECHA_CORTE]
print(f"Datos de entrenamiento hasta: {df_entrenamiento['ds'].max().date()}")

# 3. Configurar y entrenar el modelo
print("--- 3. Entrenando el modelo ---")
fcst = MLForecast(
    models=LGBMRegressor(random_state=42, n_estimators=100),
    freq='MS',
    lags=[1, 2, 3, 6, 12],
    date_features=['month', 'year'],
)
# El modelo aprende de TODAS las columnas en df_entrenamiento, incluyendo las de Fourier
fcst.fit(df_entrenamiento, static_features=[])
print("Modelo entrenado.")

# 4. Crear el DataFrame de Features Futuros (X_df) - ¡EL PASO CLAVE QUE FALTABA!
print("--- 4. Creando features futuros para la predicción ---")
horizonte_prediccion = 2
last_date = df_entrenamiento['ds'].max()
future_dates = pd.date_range(last_date, periods=horizonte_prediccion + 1, freq='MS')[1:]

# Creamos el DataFrame futuro con las columnas 'unique_id' y 'ds'
future_df = pd.DataFrame({
    'unique_id': np.repeat(df_entrenamiento['unique_id'].unique(), horizonte_prediccion),
    'ds': np.tile(future_dates, df_entrenamiento['unique_id'].nunique())
})
# AJUSTE 3: RECONSTRUIMOS las columnas categóricas en el DataFrame futuro
print("--- Reconstruyendo features categóricos en el DataFrame futuro ---")
future_df['product_id'] = future_df['unique_id'].str.split('_').str[0].astype('category')

# Calculamos los features de Fourier para esas fechas futuras
future_fourier_features = fourier(df=future_df, freq='MS', season_length=12, k=12, time_col='ds')
# Unimos los features al DataFrame futuro. Ahora tiene la misma estructura que los datos de entrenamiento.
future_df = pd.concat([future_df, future_fourier_features[0]], axis=1)
future_df = future_df.loc[:, ~future_df.columns.duplicated()]

# 5. Predecir usando el horizonte (h) y el DataFrame de features futuros (X_df)
print("--- 5. Realizando la predicción ---")
predicciones_simulacion = fcst.predict(h=horizonte_prediccion, X_df=future_df)

# 6. Comparar la predicción con los valores reales
print("--- 6. Comparando resultados ---")
resultados_validacion = pd.merge(
    df_validacion,
    predicciones_simulacion,
    on=['unique_id', 'ds']
)
print("\n--- Comparación: Valor Real vs. Predicción (Simulación) ---")
# Mostramos solo las columnas relevantes para la comparación
print(resultados_validacion[['unique_id', 'ds', 'y', 'LGBMRegressor']].rename(columns={'y': 'Valor_Real', 'LGBMRegressor': 'Prediccion'}))

--- 1. Creando features de Fourier ---
Datos de entrenamiento hasta: 2019-10-01
--- 3. Entrenando el modelo ---
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.196680 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2476
[LightGBM] [Info] Number of data points in the train set: 5781710, number of used features: 32
[LightGBM] [Info] Start training from score 0.114956
Modelo entrenado.
--- 4. Creando features futuros para la predicción ---
--- Reconstruyendo features categóricos en el DataFrame futuro ---
--- 5. Realizando la predicción ---
--- 6. Comparando resultados ---

--- Comparación: Valor Real vs. Predicc

In [ ]:
# ==============================================================================
# PARTE 1: SIMULACIÓN Y VALIDACIÓN (Predecir 201912 con datos hasta 201910)
# ==============================================================================
print("\n" + "="*60)
print(" PARTE 1: SIMULACIÓN Y VALIDACIÓN")
print("="*60)


# 3. Dividimos los datos: entrenamiento hasta 2019-10, validación después.
FECHA_CORTE = '2019-10-01'
# Elimina columnas duplicadas antes de dividir en entrenamiento y validación
df_final = df_final.loc[:, ~df_final.columns.duplicated()]

df_entrenamiento = df_final[df_final['ds'] <= FECHA_CORTE]
df_validacion = df_final[df_final['ds'] > FECHA_CORTE]

print(f"\nDatos de entrenamiento: {df_entrenamiento.shape[0]} filas, hasta la fecha {df_entrenamiento['ds'].max().date()}")
print(f"Datos de validación: {df_validacion.shape[0]} filas, desde la fecha {df_validacion['ds'].min().date()}")

# 4. Configurar y entrenar el modelo MLForecast
#    - lags: Usamos lags de hasta 12 meses para capturar estacionalidad anual.
#      Es crucial tener lags >= al horizonte de predicción (h).
#    - lag_transforms: Creamos medias móviles para suavizar la serie.
#    - date_features: El mes y el año son características muy útiles.
# Mejor combinación de hiperparámetros encontrada (ejemplo, debes ajustar según validación real)

fcst = MLForecast(
    models=LGBMRegressor(random_state=42, n_estimators=100),
    freq='MS',
    lags=[1, 2, 3, 6, 12],
    date_features=['month', 'year'],
)
# Entrenamos el modelo SOLO con los datos de entrenamiento
fcst.fit(df_entrenamiento, static_features=[])

# 5. Predecir 2 pasos hacia adelante (h=2) para obtener 201911 y 201912
# MLForecast sabe que debe empezar a predecir después de la última fecha de entrenamiento.
horizonte_prediccion = 2

predicciones_simulacion = fcst.predict(h=horizonte_prediccion)

# 6. Comparar la predicción con los valores reales
# Unimos las predicciones con los datos reales de validación para comparar.
resultados_validacion = pd.merge(
    df_validacion,
    predicciones_simulacion,
    on=['unique_id', 'ds']
)
print("\n--- Comparación: Valor Real vs. Predicción (Simulación) ---")
print(resultados_validacion.rename(columns={'y': 'Valor_Real', 'LGBMRegressor': 'Prediccion'}))




 PARTE 1: SIMULACIÓN Y VALIDACIÓN

Datos de entrenamiento: 8935370 filas, hasta la fecha 2019-10-01
Datos de validación: 525610 filas, desde la fecha 2019-11-01
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.156060 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 5781710, number of used features: 7
[LightGBM] [Info] Start training from score 0.114956

--- Comparación: Valor Real vs. Predicción (Simulación) ---
          unique_id         ds  Valor_Real  Prediccion
0       20001_10001 2019-11-01   236.65556   85.167816
1       20001_10001 2019-12-01   180.21938   80.003993
2       20001_10002 2019-11-01    45.61495   14.402763
3       20001_10002 2019-12-01   113.33165   26.585248
4       20001_10003 2019-11-01    86.14415   53.356966
...             ...        ...         .

In [ ]:
# # ==============================================================================
# # Optimización de Hiperparámetros con Optuna
# # ==============================================================================
# print("\n" + "="*60)
# print(" PARTE 1: SIMULACIÓN Y VALIDACIÓN")
# print("="*60)

# # 3. Dividimos los datos: entrenamiento hasta 2019-10, validación después.
# FECHA_CORTE = '2019-10-01'
# df_entrenamiento = df_final[df_final['ds'] <= FECHA_CORTE]
# df_validacion = df_final[df_final['ds'] > FECHA_CORTE]

# horizonte_prediccion = 2

# print(f"\nDatos de entrenamiento: {df_entrenamiento.shape[0]} filas, hasta la fecha {df_entrenamiento['ds'].max().date()}")
# print(f"Datos de validación: {df_validacion.shape[0]} filas, desde la fecha {df_validacion['ds'].min().date()}")

# def objective(trial):
#     params = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
#         'num_leaves': trial.suggest_int('num_leaves', 20, 200),
#         'max_depth': trial.suggest_int('max_depth', 3, 16),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'random_state': 42
#     }
#     model = LGBMRegressor(**params)
#     fcst = MLForecast(
#         models=[model],
#         freq='MS',
#         lags=[1, 2, 3, 6, 12],
#         lag_transforms={
#             1: [(rolling_mean, 3), (rolling_mean, 6)],
#             3: [(rolling_mean, 3)]
#         },
#         date_features=['month', 'year', 'dayofweek']
#     )
#     # Entrenamiento y validación
#     fcst.fit(df_entrenamiento)
#     preds = fcst.predict(h=horizonte_prediccion)
#     merged = pd.merge(df_validacion, preds, on=['unique_id', 'ds'])
#     score = mean_absolute_error(merged['y'], merged['LGBMRegressor'])
#     return score

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=30)

# print("Mejores hiperparámetros encontrados:")
# print(study.best_params)


In [17]:
resultados_validacion.head()

,unique_id,ds,y,sin1_12,sin2_12,sin3_12,sin4_12,sin5_12,sin6_12,sin7_12,...,cos4_12,cos5_12,cos6_12,cos7_12,cos8_12,cos9_12,cos10_12,cos11_12,cos12_12,LGBMRegressor
0,20001_10001,2019-11-01,236.65556,-4.999990e-01,-8.660243e-01,-1.000000e+00,-8.660276e-01,-4.999982e-01,-6.636076e-06,0.500006,...,-0.499996,-0.866026,-1.0,-0.866022,-0.500008,0.000005,0.500004,0.866027,1.0,52.923027
1,20001_10001,2019-12-01,180.21938,4.769952e-08,9.539905e-08,1.430986e-07,1.907981e-07,2.384976e-07,2.861971e-07,-0.000007,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,71.890963
2,20001_10002,2019-11-01,45.61495,-4.999990e-01,-8.660243e-01,-1.000000e+00,-8.660276e-01,-4.999982e-01,-6.636076e-06,0.500006,...,-0.499996,-0.866026,-1.0,-0.866022,-0.500008,0.000005,0.500004,0.866027,1.0,23.000294
3,20001_10002,2019-12-01,113.33165,4.769952e-08,9.539905e-08,1.430986e-07,1.907981e-07,2.384976e-07,2.861971e-07,-0.000007,...,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,26.630187
4,20001_10003,2019-11-01,86.14415,-4.999990e-01,-8.660243e-01,-1.000000e+00,-8.660276e-01,-4.999982e-01,-6.636076e-06,0.500006,...,-0.499996,-0.866026,-1.0,-0.866022,-0.500008,0.000005,0.500004,0.866027,1.0,25.155393


In [14]:
# 1. Extraer los valores reales y las predicciones del DataFrame
y_real = resultados_validacion['y']
y_pred = resultados_validacion['LGBMRegressor']

# 2. Calcular las métricas
mae = mean_absolute_error(y_real, y_pred)
mse = mean_squared_error(y_real, y_pred) # Calculamos el MSE primero
rmse = np.sqrt(mse) # Luego calculamos su raíz cuadrada para obtener el RMSE
mape = mean_absolute_percentage_error(y_real, y_pred)
r2 = r2_score(y_real, y_pred)


# 3. Imprimir los resultados de forma clara
print("\n" + "="*40)
print(" MÉTRICAS DE RENDIMIENTO DEL MODELO")
print("="*40)
print(f"Error Absoluto Medio (MAE):       {mae:.2f} unidades")
print(f"Raíz del Error Cuadrático (RMSE): {rmse:.2f} unidades")
print(f"Error Porcentual Absoluto (MAPE): {mape:.2%}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")
print("="*40)

print("\nInterpretación:")
print(f"- En promedio, el modelo se equivoca en {mae:.2f} toneladas (o la unidad que estés usando).")
print(f"- El error porcentual promedio es de {mape:.2%}.")
print(f"- Un R² de {r2:.2f} indica qué proporción de la varianza de los datos es explicada por el modelo (más cercano a 1 es mejor).")


 MÉTRICAS DE RENDIMIENTO DEL MODELO
Error Absoluto Medio (MAE):       0.11 unidades
Raíz del Error Cuadrático (RMSE): 1.31 unidades
Error Porcentual Absoluto (MAPE): 15300804610616704.00%
Coeficiente de Determinación (R²): 0.40

Interpretación:
- En promedio, el modelo se equivoca en 0.11 toneladas (o la unidad que estés usando).
- El error porcentual promedio es de 15300804610616704.00%.
- Un R² de 0.40 indica qué proporción de la varianza de los datos es explicada por el modelo (más cercano a 1 es mejor).


In [15]:

# ==============================================================================
# PARTE 2: APLICACIÓN REAL (Predecir 202002 con datos hasta 201912)
# ==============================================================================
print("\n" + "="*60)
print(" PARTE 2: PREDICCIÓN FINAL PARA 202002")
print("="*60)

# 7. Entrenar el modelo con TODOS los datos disponibles hasta 2019-12
# Usamos el DataFrame completo 'df_final' que contiene todos los datos.
print(f"\nRe-entrenando el modelo con todos los datos ({df_final.shape[0]} filas) hasta {df_final['ds'].max().date()}...")

# Re-entrenamos el mismo modelo (o uno nuevo con la misma config) con todos los datos
# para obtener la mejor predicción posible.
fcst.fit(df_final)

# 8. Predecir 2 meses hacia el futuro para obtener 202001 y 202002
predicciones_finales = fcst.predict(h=2)

print("\n--- ¡PREDICCIÓN FINAL PARA 202001 Y 202002! ---")
print(predicciones_finales)

# Filtramos para ver el resultado que te interesa
prediccion_target = predicciones_finales[predicciones_finales['ds'] == '2020-02-01']
print("\n--- Valor predicho específicamente para 2020-02 ---")
print(prediccion_target)



 PARTE 2: PREDICCIÓN FINAL PARA 202002

Re-entrenando el modelo con todos los datos (9460980 filas) hasta 2019-12-01...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.225444 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1291
[LightGBM] [Info] Number of data points in the train set: 6307320, number of used features: 7
[LightGBM] [Info] Start training from score 0.114459

--- ¡PREDICCIÓN FINAL PARA 202001 Y 202002! ---
          unique_id         ds  LGBMRegressor
0       20001_10001 2020-01-01      82.669177
1       20001_10001 2020-02-01      78.726539
2       20001_10002 2020-01-01      15.585885
3       20001_10002 2020-02-01      14.842661
4       20001_10003 2020-01-01      82.669177
...             ...        ...            ...
525605  21276_10462 2020-02-01       0.010059
525606  21276_10495 2020-01-01       0.007369
525607  21276_10495 2020-02-01       0.010059
525608  21276_10550 2020-01

In [16]:
prediccion_target.head()

,unique_id,ds,LGBMRegressor
1,20001_10001,2020-02-01,78.726539
3,20001_10002,2020-02-01,14.842661
5,20001_10003,2020-02-01,78.726539
7,20001_10004,2020-02-01,78.726539
9,20001_10005,2020-02-01,12.762514


In [17]:
# Separar product_id de unique_id y sumar LGBMRegressor por product_id
df_pred_sum = prediccion_target.copy()
df_pred_sum['product_id'] = df_pred_sum['unique_id'].str.split('_').str[0].astype(int)
df_pred_sum_grouped = df_pred_sum.groupby('product_id', as_index=False)['LGBMRegressor'].sum()
df_pred_sum_grouped.head()

,product_id,LGBMRegressor
0,20001,1009.024483
1,20002,666.042483
2,20003,747.234118
3,20004,544.151167
4,20005,544.287563


In [18]:
# Renombrar la columna y exportar a CSV
df_pred_sum_grouped.rename(columns={'LGBMRegressor': 'tn'}, inplace=True)
df_pred_sum_grouped.to_csv('prediccion_tn_por_producto_2.csv', index=False)